In [85]:
import pandas as pd

In [86]:
train=pd.read_csv('data/fraudTrain.csv',index_col=None)
test=pd.read_csv('data/fraudTest.csv',index_col=None)

In [87]:
## we will use 23 months for training and 1 month for testing 
## we will have some static feats
## we will have some dynamic feats for each time shots
##

In [88]:
total_data=pd.concat((train,test))
total_data.head(1)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0


In [89]:
total_data.trans_date_trans_time.max(),total_data.trans_date_trans_time.min()

('2020-12-31 23:59:34', '2019-01-01 00:00:18')

In [90]:
def get_discrete_time(time):
    year=int(time[:4])
    month=int(time[5:7])
    if year>2019:
        month+=12
    return month

In [91]:
total_data['discrete_time']=total_data['trans_date_trans_time'].apply(lambda x: get_discrete_time(x) )

In [92]:
total_data.discrete_time.max(),total_data.discrete_time.min()

(24, 1)

In [93]:
##transductive setting 

train_data=total_data[total_data['discrete_time']<24]
test_data=total_data[total_data['discrete_time']==24]
train_data.cc_num.nunique(),test_data.cc_num.nunique(),total_data.cc_num.nunique()

(997, 910, 999)

In [94]:
train_data.merchant.nunique(),test_data.merchant.nunique(),total_data.merchant.nunique()

(693, 693, 693)

In [95]:
test_data=test_data[test_data['cc_num'].isin(train_data.cc_num.unique())]

In [96]:
cc_num_dict=dict((j,i) for i,j in enumerate(train_data.cc_num.unique()))
merchant_dict=dict((j,i+train_data.cc_num.nunique()) for i,j in enumerate(train_data.merchant.unique()))
gender_dict=dict((j,i) for i,j in enumerate(train_data.gender.unique()))
city_dict=dict((j,i) for i,j in enumerate(train_data.city.unique()))
state_dict=dict((j,i) for i,j in enumerate(train_data.state.unique()))
street_dict=dict((j,i) for i,j in enumerate(train_data.street.unique()))
zip_dict=dict((j,i) for i,j in enumerate(train_data.zip.unique()))
merchant_cat_dict=dict((j,i) for i,j in enumerate(train_data.category.unique()))
job_dict=dict((j,i) for i,j in enumerate(train_data.job.unique()))


In [97]:
feat_cc_num_pre_df=train_data[['cc_num','gender','city','state','lat','long','zip',
'street','job','dob','city_pop'
]]

In [98]:
feat_cc_num_pre_df=feat_cc_num_pre_df.drop_duplicates()

In [99]:
feat_cc_num_pre_df['cc_id']=feat_cc_num_pre_df['cc_num'].apply(lambda x: cc_num_dict[x])
feat_cc_num_pre_df['state_id']=feat_cc_num_pre_df['state'].apply(lambda x: state_dict[x])
feat_cc_num_pre_df['city_id']=feat_cc_num_pre_df['city'].apply(lambda x: city_dict[x])
feat_cc_num_pre_df['street_id']=feat_cc_num_pre_df['street'].apply(lambda x: street_dict[x])
feat_cc_num_pre_df['job_id']=feat_cc_num_pre_df['job'].apply(lambda x: job_dict[x])
feat_cc_num_pre_df['zip_id']=feat_cc_num_pre_df['zip'].apply(lambda x: zip_dict[x])
feat_cc_num_pre_df['dob_year']=feat_cc_num_pre_df['dob'].apply(lambda x: x[:4])

In [100]:
total_data.columns

Index([u'Unnamed: 0', u'trans_date_trans_time', u'cc_num', u'merchant',
       u'category', u'amt', u'first', u'last', u'gender', u'street', u'city',
       u'state', u'zip', u'lat', u'long', u'city_pop', u'job', u'dob',
       u'trans_num', u'unix_time', u'merch_lat', u'merch_long', u'is_fraud',
       u'discrete_time'],
      dtype='object')

In [104]:
cc_num_static_feat=feat_cc_num_pre_df[['cc_id','state_id','city_id','street_id','job_id','dob_year','city_pop','zip_id','lat','long','gender']]

In [105]:
cc_num_static_feat.head(1)

,cc_id,state_id,city_id,street_id,job_id,dob_year,city_pop,zip_id,lat,long,gender
0,0,0,0,0,0,1988,3495,0,36.0788,-81.1781,F


In [121]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
encoder_df = encoder.fit_transform(cc_num_static_feat[['gender']]).toarray()

In [124]:
import numpy as np
final_cc_feat=np.concatenate((cc_num_static_feat[['cc_id','lat','long','dob_year']].values,encoder_df),axis=1)

In [171]:
final_cc_feat_df=pd.DataFrame(final_cc_feat)
final_cc_feat_df.columns=['cc_id','lat','long','dob_year','gen-1','gen-2']
final_cc_feat_df.to_csv('cc_num_feat.csv',index=None)

In [ ]:
feat_cc_num_pre_df['']

In [ ]:
train_data['cc_id']=train_data.cc_num.apply(lambda x: cc_num_dict[x])
train_data['merchant_id']=train_data.merchant.apply(lambda x:merchant_dict[x])

In [110]:
train_data['weight']=1
train_data['merchant']

C:\Users\nathp\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0                       fraud_Rippin, Kub and Mann
1                  fraud_Heller, Gutmann and Zieme
2                             fraud_Lind-Buckridge
3               fraud_Kutch, Hermiston and Farrell
4                              fraud_Keeling-Crist
5                 fraud_Stroman, Hudson and Erdman
6                            fraud_Rowe-Vandervort
7                             fraud_Corwin-Collins
8                                 fraud_Herzog Ltd
9                fraud_Schoen, Kuphal and Nitzsche
10                          fraud_Rutherford-Mertz
11                           fraud_Kerluke-Abshire
12                               fraud_Lockman Ltd
13                                 fraud_Kiehn Inc
14                               fraud_Beier-Hyatt
15                          fraud_Schmidt and Sons
16                          fraud_Lebsack and Sons
17                              fraud_Mayert Group
18        fraud_Konopelski, Schneider and Hartmann
19               fraud_Schultz,

In [111]:
train_edges=train_data[['cc_id','merchant_id','weight','discrete_time']]

In [ ]:
train_edges

In [113]:
test_data['cc_id']=test_data.cc_num.apply(lambda x: cc_num_dict[x])
test_data['merchant_id']=test_data.merchant.apply(lambda x:merchant_dict[x])

In [114]:
test_data['weight']=1
test_edges=test_data[['cc_id','merchant_id','weight','discrete_time']]


In [120]:
train_edges.to_csv('./kag_trans_train.csv',index=None)
test_edges.to_csv('./kag_trans_test.csv',index=None)

In [129]:
feat_merc_num_pre_df=train_data[['merchant','category','merch_lat','merch_long']]

In [149]:
merch_feat.columns,max_merc_feat.columns

(Index([u'category_x', u'merch_lat_x', u'merch_long_x', u'category_y',
        u'merch_lat_y', u'merch_long_y'],
       dtype='object'),
 Index([u'category', u'merch_lat', u'merch_long'], dtype='object'))

In [150]:
max_merc_feat=feat_merc_num_pre_df.groupby(['merchant'],as_index=False).max()
min_merc_feat=feat_merc_num_pre_df.groupby(['merchant'],as_index=False).min()

In [154]:
merch_feat=pd.merge(max_merc_feat,min_merc_feat,on='merchant',how='left')
merch_feat['merchant_id']=merch_feat.merchant.apply(lambda x: merchant_dict[x])

merch_feat['category_id']=merch_feat.category_x.apply(lambda x: merchant_cat_dict[x])

In [164]:
merch_feat.category_id.value_counts()

13    50
12    50
11    50
9     50
7     50
6     50
5     50
3     50
2     50
1     50
8     49
0     49
10    48
4     47
Name: category_id, dtype: int64

In [165]:
encoder_df = encoder.fit_transform(merch_feat[['category_id']]).toarray()

In [167]:
merch_final_feat=np.concatenate((merch_feat[['merchant_id','merch_lat_x','merch_long_x','merch_lat_y','merch_long_y']],encoder_df),axis=1)

In [168]:
merch_final_feat_df=pd.DataFrame(merch_final_feat)
merch_final_feat_df.columns=['merchant_id','merch_lat_x','merch_long_x','merch_lat_y','merch_long_y',
'1','2','3','4','5','6','7','8','9','10','11','12','13','14']

In [169]:
merch_final_feat_df

,merchant_id,merch_lat_x,merch_long_x,merch_lat_y,merch_long_y,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1508.0,65.764956,-67.046553,19.303454,-166.398869,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1421.0,65.743344,-67.010222,19.275133,-166.045568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1642.0,65.441387,-68.002109,19.378039,-166.651656,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1152.0,65.338913,-67.626967,19.496400,-166.366034,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1096.0,65.441336,-68.688986,20.754155,-156.847401,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1620.0,65.454020,-67.040593,19.759984,-166.659277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,1482.0,67.510267,-68.131216,19.144328,-166.017543,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1575.0,66.554992,-66.963918,19.057104,-165.649037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,1068.0,65.798838,-66.996422,19.537941,-166.657834,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1543.0,65.638985,-67.759070,20.983422,-165.420090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [170]:
merch_final_feat_df.to_csv('./merchant_feat.csv',index=None)

In [132]:
train_data.merchant.nunique()

693

In [130]:
feat_merc_num_pre_df=feat_merc_num_pre_df.drop_duplicates()

In [131]:
feat_merc_num_pre_df

,merchant,category,merch_lat,merch_long
0,"fraud_Rippin, Kub and Mann",misc_net,36.011293,-82.048315
1,"fraud_Heller, Gutmann and Zieme",grocery_pos,49.159047,-118.186462
2,fraud_Lind-Buckridge,entertainment,43.150704,-112.154481
3,"fraud_Kutch, Hermiston and Farrell",gas_transport,47.034331,-112.561071
4,fraud_Keeling-Crist,misc_pos,38.674999,-78.632459
5,"fraud_Stroman, Hudson and Erdman",gas_transport,40.653382,-76.152667
6,fraud_Rowe-Vandervort,grocery_net,37.162705,-100.153370
7,fraud_Corwin-Collins,gas_transport,38.948089,-78.540296
8,fraud_Herzog Ltd,misc_pos,40.351813,-79.958146
9,"fraud_Schoen, Kuphal and Nitzsche",grocery_pos,37.179198,-87.485381


In [116]:
train_edges.cc_id.max(),train_edges.merchant_id.min()

(996, 997)

In [119]:
train_edges.shape,test_edges.shape,total_data.shape

((1712856, 4), (139514, 4), (1852394, 24))

In [118]:
train_edges

,cc_id,merchant_id,weight,discrete_time
0,0,997,1,1
1,1,998,1,1
2,2,999,1,1
3,3,1000,1,1
4,4,1001,1,1
5,5,1002,1,1
6,6,1003,1,1
7,7,1004,1,1
8,8,1005,1,1
9,9,1006,1,1


In [ ]:
import torch
import pandas as pd

: 